In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [2]:
torch.cuda.set_device(4)

In [3]:
dbunch = get_3d_dbunch_aug(Meta.df_any, bs=10)

In [4]:
x,y = dbunch.one_batch()
x.shape, y.shape

(torch.Size([10, 60, 3, 256, 256]), torch.Size([10, 60, 6]))

## Model

In [5]:
m = get_3d_head()

In [6]:
config=dict(custom_head=m)
learn = get_learner(dbunch, xresnet18, get_loss(), config=config)

In [7]:
hook = ReshapeBodyHook(learn.model[0])
learn.add_cb(DePadLoss())

In [9]:
learn.load(f'runs/baseline_any_xresnet18-3-full', strict=True)

RuntimeError: Error(s) in loading state_dict for Sequential:
	Missing key(s) in state_dict: "0.4.0.convpath.0.0.weight", "0.4.0.convpath.0.1.weight", "0.4.0.convpath.0.1.bias", "0.4.0.convpath.0.1.running_mean", "0.4.0.convpath.0.1.running_var", "0.4.0.convpath.1.0.weight", "0.4.0.convpath.1.1.weight", "0.4.0.convpath.1.1.bias", "0.4.0.convpath.1.1.running_mean", "0.4.0.convpath.1.1.running_var", "0.4.1.convpath.0.0.weight", "0.4.1.convpath.0.1.weight", "0.4.1.convpath.0.1.bias", "0.4.1.convpath.0.1.running_mean", "0.4.1.convpath.0.1.running_var", "0.4.1.convpath.1.0.weight", "0.4.1.convpath.1.1.weight", "0.4.1.convpath.1.1.bias", "0.4.1.convpath.1.1.running_mean", "0.4.1.convpath.1.1.running_var", "0.5.0.convpath.0.0.weight", "0.5.0.convpath.0.1.weight", "0.5.0.convpath.0.1.bias", "0.5.0.convpath.0.1.running_mean", "0.5.0.convpath.0.1.running_var", "0.5.0.convpath.1.0.weight", "0.5.0.convpath.1.1.weight", "0.5.0.convpath.1.1.bias", "0.5.0.convpath.1.1.running_mean", "0.5.0.convpath.1.

In [9]:
learn.model

Sequential(
  (0): Sequential(
    (0): ConvLayer(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): ConvLayer(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): ConvLayer(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): ResBlock(
        (convpath): Sequential(
          (0): ConvLayer(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): Ba

In [10]:
# learn.summary()

## Training

In [11]:
name = 'train3d_baseline'

In [ ]:
# learn.lr_find()

In [12]:
do_fit(learn, 6, 1e-2, freeze=False)
learn.save(f'runs/{name}-1')

KeyboardInterrupt: 